<a href="https://colab.research.google.com/github/jjwjjwjjwjjwjjw/Recommender-System/blob/main/2.%20%EC%83%81%ED%92%88_%EC%9D%B4%EB%AF%B8%EC%A7%80%EB%A5%BC_%EC%A0%81%EC%9A%A9%ED%95%98%EC%97%AC_%EC%B6%94%EC%B2%9C%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%EC%8B%A4%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 Feature vector를 활용해보기!

- 여러 논문에서 소개된것처럼 pre-trained CNN으로 image features를 추출해봅니다.
- image features를 비교하여 실제로 비슷한지 판단하고, rating정보와 함께 분석해봅니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/data/amazon_reviews'

## Load dataset

1. AMAZON_FASHION_5.json
2. All_Beauty_5.json
3. Luxury_Beauty_5.json

In [ ]:
import os, json
import pandas as pd

In [ ]:
def load_json(filename):
  data = []
  with open(os.path.join(path, filename), 'r', encoding='utf-8') as f:
      for l in f:
        data.append(json.loads(l.strip()))

  df = pd.DataFrame.from_dict(data)

  # get rows which contains image
  df = df[~df['image'].isnull()]

  return df

In [ ]:
fashion_df = load_json('AMAZON_FASHION_5.json')
print(fashion_df.shape)
print(fashion_df.head())

(106, 12)
     overall  verified   reviewTime      reviewerID        asin  \
164      5.0      True  04 18, 2018  A2YZERYQTLB8NG  B001IKJOLW   
172      5.0      True   04 7, 2018  A1CKPC88NHMYGR  B001IKJOLW   
179      5.0      True  03 22, 2018  A3KKVVAINMZF9D  B001IKJOLW   
192      5.0      True  02 27, 2018  A3TLWN2BRF1QH5  B001IKJOLW   
197      5.0      True  02 20, 2018  A3RNGBSBRJ3YAQ  B001IKJOLW   

                                                 style      reviewerName  \
164  {'Size:': ' 9.5 B(M) US', 'Color:': ' Black/Wh...             Lenci   
172  {'Size:': ' 11 B(M) US', 'Color:': ' Wolf Grey...     Cynthia Foyer   
179  {'Size:': ' 9 B(M) US', 'Color:': ' Blue Tint/...  Nadege Marcellus   
192  {'Size:': ' 8.5 B(M) US', 'Color:': ' Blue Tin...       Brittany C.   
197  {'Size:': ' 7.5 B(M) US', 'Color:': ' Wolf Gre...    Andrea L Hogan   

                                            reviewText  \
164  Best tennis shoes I've had all my life. Very c...   
172           

In [ ]:
beauty_df = load_json('All_Beauty_5.json')
print(beauty_df.shape)
print(beauty_df.head())

(98, 12)
    overall  verified   reviewTime      reviewerID        asin  \
19      5.0      True  04 23, 2018   AX0ZEGHH0H525  B00006L9LC   
20      5.0      True  04 22, 2018  A1L0QECT7J93ZP  B00006L9LC   
21      5.0      True  04 21, 2018  A1VN560NNZQIR0  B00006L9LC   
34      1.0      True  03 27, 2018  A2V608ILSK1M5R  B00006L9LC   
47      5.0      True  02 23, 2018  A22V1MD93T2FW9  B00006L9LC   

                  style   reviewerName  \
19  {'Size:': ' Small'}         Aida A   
20  {'Size:': ' Small'}          Elena   
21  {'Size:': ' Small'}     Shablinska   
34  {'Size:': ' Small'}       CDART815   
47  {'Size:': ' Small'}  Heather Sharp   

                                           reviewText  \
19  Suffered from itchiness under my hair for coup...   
20  Got this product for me and  my daughter. I ca...   
21  Cleansing properties are above any praise! Sup...   
34  My product was not sealed and either used or s...   
47  I bought this for my husband. Hed been having ...   

In [ ]:
luxury_df = load_json('Luxury_Beauty_5.json')
print(luxury_df.shape)
print(luxury_df.head())

(617, 12)
     overall  verified   reviewTime      reviewerID        asin  \
68       5.0      True   03 5, 2018   A2BHOZILR7SY9  B000142FVW   
75       5.0      True  01 13, 2018   ACMSQCH1H7JZD  B000142FVW   
86       5.0      True  10 17, 2017  A2L77YQRAEA1YZ  B000142FVW   
88       5.0      True   03 5, 2017  A28W77RPDZK7AZ  B00014351Q   
104      5.0     False  08 21, 2016  A2IV70BWQBUF32  B00014351Q   

                                           style   reviewerName  \
68             {'Color:': ' Tickle My France-y'}    MustangMary   
75                    {'Color:': ' Samoan Sand'}            KED   
86                    {'Color:': ' Samoan Sand'}           ELLE   
88   {'Color:': ' Crawfishin' for a Compliment'}         Angela   
104          {'Color:': ' Rich Girls & Po-Boys'}  Paige Sanders   

                                            reviewText  \
68   This is the neutral I was searching for this w...   
75   Very light color. I'm pale and it matches my s...   
86   Best 

## Download images

In [ ]:
from tqdm import tqdm
import requests

In [ ]:
def download_images(path, df, category):
  folder_path = os.path.join(path, category)

  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  for index in tqdm(df.index):
    url_list = df['image'].loc[index]
    for url_index, url in enumerate(url_list):
      if not os.path.exists(os.path.join(folder_path, f'{str(index)}_{str(url_index)}.jpg')):
        img_data = requests.get(url).content
        with open(os.path.join(folder_path, f'{str(index)}_{str(url_index)}.jpg'), 'wb') as handler:
          handler.write(img_data)

  print(f'{category}: {len(df.index)} images downloaded or already exist...')

In [ ]:
download_images(path, beauty_df, 'beauty')

100%|██████████| 98/98 [00:41<00:00,  2.34it/s]

beauty: 98 images downloaded or already exist...


In [ ]:
download_images(path, fashion_df, 'fashion')

100%|██████████| 106/106 [00:31<00:00,  3.38it/s]

fashion: 106 images downloaded or already exist...


In [ ]:
download_images(path, luxury_df, 'luxury')

100%|██████████| 617/617 [09:12<00:00,  1.12it/s]

luxury: 617 images downloaded or already exist...


## Use pre-trained CNN

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [ ]:
# Load the pretrained model
model = models.resnet18(pretrained=True)
# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 62.8MB/s]


In [ ]:
# Set model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [ ]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
      my_embedding.copy_(o.data.reshape(o.data.size(1)))
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding.cpu().detach().numpy()

In [ ]:
# Test
category = 'beauty'
for image_file in os.listdir(os.path.join(path, category))[:2]:
  print(f"{image_file} feature vectors")
  print(get_vector(os.path.join(path, category+'/'+image_file)))

19_0.jpg feature vectors
[3.30382913e-01 1.60150200e-01 6.71930373e-01 1.35232806e-01
 8.22356701e-01 1.87839353e+00 5.21143854e-01 5.15729785e-01
 2.70418763e-01 1.92555681e-01 1.37338430e-01 3.49327356e-01
 3.16603398e+00 7.70494640e-01 2.47499466e+00 3.87524903e-01
 1.67651191e-01 1.74052060e+00 6.53961003e-01 2.12244940e+00
 5.85476339e-01 3.10379565e-01 5.68900824e+00 4.03157294e-01
 1.44829345e+00 2.25878525e+00 8.72986794e-01 9.73813593e-01
 5.69142461e-01 4.81952548e-01 1.28070569e+00 5.04999220e-01
 2.42233321e-01 9.31035876e-02 1.43523920e+00 4.12864447e-01
 9.40229833e-01 3.83923912e+00 4.18769300e-01 6.51138842e-01
 3.70566517e-01 8.71357679e-01 1.79575098e+00 3.72599077e+00
 4.01102453e-01 9.51619387e-01 6.12748027e-01 8.93791437e-01
 7.46635973e-01 2.44181895e+00 2.45340085e+00 1.23427951e+00
 4.32807505e-01 3.04543972e+00 1.08898222e-01 1.17805386e+00
 1.03500509e+00 1.62799895e+00 1.03457904e+00 3.70118886e-01
 1.26682389e+00 1.55346012e+00 1.20078838e+00 3.11978281e-01

## Preprocess dataset

- Remove unnecessary columns
- Remove all other columns except `overall`, `reviewerID`, `asin`, `image`
- Create new column with image filename

In [ ]:
def add_image_filenames(category, df):
  # Remove unnecessary columns
  df = df[['overall','reviewerID', 'asin', 'image']]

  filenames = []
  for row_index in df.index:
    each_files = []
    for idx in range(len(df.loc[row_index]['image'])):
      each_files.append(os.path.join(path, category+'/'+f'{row_index}_{idx}.jpg'))
    filenames.append(each_files)

  # Add new column
  df.drop('image', axis=1, inplace=True)
  df['image_filename'] = list(filenames)

  return df


In [ ]:
luxury_df = add_image_filenames('luxury', luxury_df)
beauty_df = add_image_filenames('beauty', beauty_df)
fashion_df = add_image_filenames('fashion', fashion_df)

<ipython-input-21-6d060dbe43c9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('image', axis=1, inplace=True)
<ipython-input-21-6d060dbe43c9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_filename'] = list(filenames)
<ipython-input-21-6d060dbe43c9>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('image', axis=1, inplace=True)
<ipython-input-21-6

### Combine into one dataframe

- `luxury_df`, `fashion_df`, beauty_df

In [ ]:
luxury_df.head()

,overall,reviewerID,asin,image_filename
68,5.0,A2BHOZILR7SY9,B000142FVW,[/content/drive/MyDrive/data/amazon_reviews/lu...
75,5.0,ACMSQCH1H7JZD,B000142FVW,[/content/drive/MyDrive/data/amazon_reviews/lu...
86,5.0,A2L77YQRAEA1YZ,B000142FVW,[/content/drive/MyDrive/data/amazon_reviews/lu...
88,5.0,A28W77RPDZK7AZ,B00014351Q,[/content/drive/MyDrive/data/amazon_reviews/lu...
104,5.0,A2IV70BWQBUF32,B00014351Q,[/content/drive/MyDrive/data/amazon_reviews/lu...


In [ ]:
data_list = []
dataframe_list = [('luxury',luxury_df), ('beauty', beauty_df), ('fashion', fashion_df)]
for dataframe in dataframe_list:
  category = dataframe[0]
  df = dataframe[1]
  for index, row in df.iterrows():
    for filename in row['image_filename']:
      data_tuple = (category, row['overall'], row['reviewerID'], row['asin'], filename)
      data_list.append(data_tuple)

In [ ]:
combined_df = pd.DataFrame(data=data_list,columns=['category', 'overall', 'reviewerID', 'asin', 'filename'])
print(combined_df.head())
print(combined_df.shape)

  category  overall      reviewerID        asin  \
0   luxury      5.0   A2BHOZILR7SY9  B000142FVW   
1   luxury      5.0   ACMSQCH1H7JZD  B000142FVW   
2   luxury      5.0  A2L77YQRAEA1YZ  B000142FVW   
3   luxury      5.0  A28W77RPDZK7AZ  B00014351Q   
4   luxury      5.0  A28W77RPDZK7AZ  B00014351Q   

                                            filename  
0  /content/drive/MyDrive/data/amazon_reviews/lux...  
1  /content/drive/MyDrive/data/amazon_reviews/lux...  
2  /content/drive/MyDrive/data/amazon_reviews/lux...  
3  /content/drive/MyDrive/data/amazon_reviews/lux...  
4  /content/drive/MyDrive/data/amazon_reviews/lux...  
(1368, 5)


### Get image feature vectors

In [ ]:
combined_df['image_vec'] = combined_df['filename'].apply(lambda x: get_vector(x))
combined_df.to_csv(os.path.join(path, 'image_dataset.csv'), sep='\t')

In [ ]:
combined_df.head()

,category,overall,reviewerID,asin,filename,image_vec
0,luxury,5.0,A2BHOZILR7SY9,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.08537156, 0.8881109, 0.19664137, 0.0179961,..."
1,luxury,5.0,ACMSQCH1H7JZD,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[2.4444883, 0.5119337, 0.026267145, 0.09580689..."
2,luxury,5.0,A2L77YQRAEA1YZ,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[1.5468208, 0.9646034, 0.34404954, 0.6072519, ..."
3,luxury,5.0,A28W77RPDZK7AZ,B00014351Q,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.25513047, 0.13189793, 1.4722723, 1.3537146,..."
4,luxury,5.0,A28W77RPDZK7AZ,B00014351Q,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.28366375, 0.18102507, 1.2546515, 1.42258, 0..."


## K-means clustering

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
def check_vector(vector):
  return np.array([0.0 if str(x) == '' else float(x) for x in vector])[:512]

In [ ]:
df = pd.read_csv(os.path.join(path, 'image_dataset.csv'), sep='\t', index_col=0,
                 converters={"image_vec": lambda x: x.strip("[]").replace('\n','').split(" ")})


In [ ]:
df.head()

,category,overall,reviewerID,asin,filename,image_vec
0,luxury,5.0,A2BHOZILR7SY9,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[8.53715613e-02, 8.88110876e-01, 1.96641371e-0..."
1,luxury,5.0,ACMSQCH1H7JZD,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[2.44448829e+00, 5.11933684e-01, 2.62671448e-0..."
2,luxury,5.0,A2L77YQRAEA1YZ,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[1.54682076e+00, 9.64603424e-01, 3.44049543e-0..."
3,luxury,5.0,A28W77RPDZK7AZ,B00014351Q,/content/drive/MyDrive/data/amazon_reviews/lux...,"[2.55130470e-01, 1.31897926e-01, 1.47227228e+0..."
4,luxury,5.0,A28W77RPDZK7AZ,B00014351Q,/content/drive/MyDrive/data/amazon_reviews/lux...,"[2.83663750e-01, 1.81025073e-01, 1.25465155e+0..."


In [ ]:
df['image_vec'] = df['image_vec'].apply(lambda x: check_vector(x))

In [ ]:
df.head()

,category,overall,reviewerID,asin,filename,image_vec
0,luxury,5.0,A2BHOZILR7SY9,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.0853715613, 0.888110876, 0.196641371, 0.017..."
1,luxury,5.0,ACMSQCH1H7JZD,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[2.44448829, 0.511933684, 0.0262671448, 0.0958..."
2,luxury,5.0,A2L77YQRAEA1YZ,B000142FVW,/content/drive/MyDrive/data/amazon_reviews/lux...,"[1.54682076, 0.964603424, 0.344049543, 0.60725..."
3,luxury,5.0,A28W77RPDZK7AZ,B00014351Q,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.25513047, 0.131897926, 1.47227228, 1.353714..."
4,luxury,5.0,A28W77RPDZK7AZ,B00014351Q,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.28366375, 0.181025073, 1.25465155, 1.42258,..."


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=1234)
print(train_df.shape)
print(test_df.shape)

(1094, 6)
(274, 6)


In [ ]:
X_train = np.array([list(x) for x in train_df['image_vec'].values])

In [ ]:
X_train[:3]

array([[0.46087483, 0.31385082, 0.63079798, ..., 0.22194316, 0.5403595 ,
        0.14517582],
       [0.59298879, 0.22690123, 1.40304923, ..., 0.07075444, 0.61647248,
        0.49897668],
       [0.        , 1.05371392, 1.21294725, ..., 0.10871451, 0.23530663,
        1.69686472]])

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
kmeans.labels_

array([2, 1, 0, ..., 2, 1, 2], dtype=int32)

In [ ]:
test_df.head()

,category,overall,reviewerID,asin,filename,image_vec
266,luxury,5.0,A24YNYNC6QJNBN,B002B4540O,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.343839645, 1.25686955, 1.36802995, 0.318519..."
1088,beauty,5.0,A245UNW3PI53NG,B0009RF9DW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[1.69288373, 0.0861781836, 0.599285424, 0.8750..."
488,luxury,4.0,A25QBCHO0KFT0P,B00B95PWYE,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.1755649, 1.50343013, 0.321579635, 0.3567925..."
1087,beauty,5.0,A85ENSL5HBBZF,B0009RF9DW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[0.730758369, 0.275991082, 0.800452352, 0.1329..."
1152,beauty,4.0,A25QBCHO0KFT0P,B0010ZBORW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[1.60743976, 0.281613111, 2.13132811, 0.371168..."


### Evaluation

In [ ]:
# kmeans.predict([test_df['image_vec'].iloc[0]])
test_df['prediction'] = test_df['image_vec'].apply(lambda x: kmeans.predict([x])[0])
test_df.head()

,category,overall,reviewerID,asin,filename,image_vec,prediction
266,luxury,5.0,A24YNYNC6QJNBN,B002B4540O,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.343839645, 1.25686955, 1.36802995, 0.318519...",1
1088,beauty,5.0,A245UNW3PI53NG,B0009RF9DW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[1.69288373, 0.0861781836, 0.599285424, 0.8750...",1
488,luxury,4.0,A25QBCHO0KFT0P,B00B95PWYE,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.1755649, 1.50343013, 0.321579635, 0.3567925...",2
1087,beauty,5.0,A85ENSL5HBBZF,B0009RF9DW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[0.730758369, 0.275991082, 0.800452352, 0.1329...",1
1152,beauty,4.0,A25QBCHO0KFT0P,B0010ZBORW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[1.60743976, 0.281613111, 2.13132811, 0.371168...",2


In [ ]:
test_df.groupby('category')['prediction'].count()

category
beauty      35
fashion     28
luxury     211
Name: prediction, dtype: int64

In [ ]:
test_df.groupby('category').count()

,overall,reviewerID,asin,filename,image_vec,prediction
category,,,,,,
beauty,35,35,35,35,35,35
fashion,28,28,28,28,28,28
luxury,211,211,211,211,211,211


In [ ]:
test_df.groupby('prediction').count()

,category,overall,reviewerID,asin,filename,image_vec
prediction,,,,,,
0,68,68,68,68,68,68
1,95,95,95,95,95,95
2,111,111,111,111,111,111


In [ ]:
print(test_df[(test_df.prediction == 0) & (test_df.category == 'luxury')].shape)
print(test_df[(test_df.prediction == 1) & (test_df.category == 'luxury')].shape)
print(test_df[(test_df.prediction == 2) & (test_df.category == 'luxury')].shape)

(55, 7)
(69, 7)
(87, 7)


## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)

- `category`가 아닌 `overall`로 간단하게 평점 예측하기

In [ ]:
y_train = train_df['overall'].values
y_train

array([4., 4., 5., ..., 5., 4., 5.])

In [ ]:
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
test_df['prediction'] = test_df['image_vec'].apply(lambda x: neigh.predict([x])[0])

In [ ]:
test_df.head()

,category,overall,reviewerID,asin,filename,image_vec,prediction
266,luxury,5.0,A24YNYNC6QJNBN,B002B4540O,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.343839645, 1.25686955, 1.36802995, 0.318519...",5.0
1088,beauty,5.0,A245UNW3PI53NG,B0009RF9DW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[1.69288373, 0.0861781836, 0.599285424, 0.8750...",5.0
488,luxury,4.0,A25QBCHO0KFT0P,B00B95PWYE,/content/drive/MyDrive/data/amazon_reviews/lux...,"[0.1755649, 1.50343013, 0.321579635, 0.3567925...",4.0
1087,beauty,5.0,A85ENSL5HBBZF,B0009RF9DW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[0.730758369, 0.275991082, 0.800452352, 0.1329...",5.0
1152,beauty,4.0,A25QBCHO0KFT0P,B0010ZBORW,/content/drive/MyDrive/data/amazon_reviews/bea...,"[1.60743976, 0.281613111, 2.13132811, 0.371168...",4.0


In [ ]:
test_df[test_df.overall == test_df.prediction].count()

category      167
overall       167
reviewerID    167
asin          167
filename      167
image_vec     167
prediction    167
dtype: int64

In [ ]:
len(test_df)

274

In [ ]:
174/274

0.635036496350365